# 1. Dask cluster starting with jupyterhub authentification works
## 1.1 Test done to create, attach, dettach the cluster from 'same' user, and ca shut them down. 
**todo would this work with EGI checkin created jupyterhub-daskhub?


Configuration tested with
https://147.251.21.10/


In [1]:
from dask_gateway import Gateway
gateway=Gateway()

Clean up existing clouster if there is any

In [5]:
clusters=gateway.list_clusters()
for cluster in clusters:
    cluster = gateway.connect(cluster.name)
    cluster.shutdown()

In [6]:
gateway.list_clusters()

[]

Create new cluster


In [7]:
cluster = gateway.new_cluster()

In [8]:
cluster.scale(2)

In [9]:
from distributed import Client
client=Client(cluster)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+-------------+----------+-----------+---------+
| Package     | client   | scheduler | workers |
+-------------+----------+-----------+---------+
| dask        | 2022.7.0 | 2022.6.0  | None    |
| distributed | 2022.7.0 | 2022.6.0  | None    |
| lz4         | 4.0.0    | None      | None    |
| toolz       | 0.12.0   | 0.11.2    | None    |
+-------------+----------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [10]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/daskhub.2f8f65196be5416ba368161fc277966b/status,


## 1.2 Simple test of dask (without xarray) works


In [11]:
from dask import delayed
import numpy as np
f=delayed(lambda x: np.exp(x*x))

#client.submit(f(2))
client.compute(f(2))

<Future: pending, key: lambda-4ef9fcc2-0cbd-45d9-a02e-5b5861d00913>

In [12]:
import dask
dask.compute(f(2))

(54.598150033144236,)

# 2. Worker Option setting for daskhub

## 2.1 We should be able to set worker's size, but we can not.  

In [13]:
options = gateway.cluster_options()
options

In [14]:
cluster = gateway.new_cluster(worker_cores=2)

ValueError: Unknown fields ['worker_cores']

## 2.2 Worker Option setting for daskhub how can we set?


This is probably because, the serverconfiguration necesary for this (https://gateway.dask.org/cluster-options.html#server-configuration) is not setted?

may be the configuration found in this issue https://github.com/2i2c-org/infrastructure/issues/291 around dask_gateway_server.options

or https://github.com/pangeo-data/pangeo-cloud-federation/blob/80dae15bb0e7b70f69fa290c1d2ee385ccbd2d83/pangeo-deploy/values.yaml#L68

or https://github.com/zonca/jupyterhub-deploy-kubernetes-jetstream/blob/master/dask_gateway/config_dask-gateway.yaml

might help?

# 3 image of dask workers may be not mounting same NFS server as jupyter hub? & may be not using same image as jupyter hub

## 3.1 example without dask of xarray works. 

In [15]:
import xarray as xr


In [16]:
ds=xr.tutorial.open_dataset("air_temperature")# ,chunks={"time":500})

In [17]:
ds.compute()

<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## 3.2 Example with dask fail

In [18]:
ds=xr.tutorial.open_dataset("air_temperature",chunks={"time":500})

In [19]:
ds=ds.persist()
ds.compute()

KilledWorker: ('original-open_dataset-04ded617e558e7b786d9c4e99ad7e24bair-f3693454c311a082d8ea198df452a203', <WorkerState 'tls://10.244.1.43:36979', name: dask-worker-2f8f65196be5416ba368161fc277966b-lgdqq, status: closed, memory: 0, processing: 2>)

## 3.3  What is the enviroment of dask worker?

In [20]:
client.get_versions(check=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+-------------+----------------+----------------+----------------+
| Package     | client         | scheduler      | workers        |
+-------------+----------------+----------------+----------------+
| dask        | 2022.7.0       | 2022.6.0       | 2022.6.0       |
| distributed | 2022.7.0       | 2022.6.0       | 2022.6.0       |
| lz4         | 4.0.0          | None           | None           |
| pandas      | 1.4.3          | None           | None           |
| python      | 3.9.13.final.0 | 3.10.5.final.0 | 3.10.5.final.0 |
| toolz       | 0.12.0         | 0.11.2         | 0.11.2         |
+-------------+----------------+----------------+----------------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None}},
 'workers': {'tls://10.244.1.42:34903': {'host': {'python': '3.10.5.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '5.4.0-109-generic',
    'machine': 'x86_64',
    'processor': '',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.5.final.0',
    'dask': '2022.6.0',
    'distributed': '2022.6.0',
    'msgpack': '1.0.4',
    'cloudpickle': '2.1.0',
    'tornado': '6.1',
    'toolz': '0.11.2',
    'numpy': '1.22.4',
    'pandas

## 3.3 enviroment of this notebook


In [30]:
%conda list 

# packages in environment at /srv/conda/envs/notebook:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
abseil-cpp                20210324.2           h9c3ff4c_0    conda-forge
absl-py                   1.1.0              pyhd8ed1ab_0    conda-forge
adal                      1.2.7              pyhd8ed1ab_0    conda-forge
adlfs                     2022.7.0           pyhd8ed1ab_0    conda-forge
affine                    2.3.1              pyhd8ed1ab_0    conda-forge
aiobotocore               2.3.4              pyhd8ed1ab_0    conda-forge
aiohttp                   3.8.1            py39hb9d737c_1    conda-forge
aioitertools              0.10.0             pyhd8ed1ab_0    conda-forge
aiosignal                 1.2.0              pyhd8ed1ab_0    conda-forge
alembic                   1.8.0              pyhd8ed1ab_0    conda-forge


## 3.4 How to install new packages

In [23]:
from distributed.diagnostics.plugin import PipInstall

In [24]:
extra_packages=["xarray","netCDF4","pooch","h5netcdf"]
plugin=PipInstall(extra_packages,restart=True)

In [25]:
client.register_worker_plugin(plugin)

{'tls://10.244.1.42:34903': {'status': 'OK'},
 'tls://10.244.1.43:34249': {'status': 'OK'}}

In [26]:
client.get_versions(packages=extra_packages,check=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+-------------+----------+-----------+---------+
| Package     | client   | scheduler | workers |
+-------------+----------+-----------+---------+
| dask        | 2022.7.0 | 2022.6.0  | None    |
| distributed | 2022.7.0 | 2022.6.0  | None    |
| lz4         | 4.0.0    | None      | None    |
| toolz       | 0.12.0   | 0.11.2    | None    |
+-------------+----------+-----------+---------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None,
   'xarray': None,
   'netCDF4': None,
   'pooch': None,
   'h5netcdf': None}},
 'workers': {},
 'client': {'host': {'python': '3.9.13.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'C.UTF-8',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.9.13.final.0',
   'dask': '2022.7.0',
   'distributed': '2022.7.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1

In [29]:
ds.air.mean(dim='time').persist().compute()

KilledWorker: ('original-open_dataset-04ded617e558e7b786d9c4e99ad7e24bair-f3693454c311a082d8ea198df452a203', <WorkerState 'tls://10.244.1.43:36979', name: dask-worker-2f8f65196be5416ba368161fc277966b-lgdqq, status: closed, memory: 0, processing: 2>)

In [29]:
url='https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'
ds=xr.open_dataset(url,engine="netcdf4")
#ds.compute()#.mean(dim="time").compute()#.air.data.visualize()
#ds=ds.chunk({"time":500})


OSError: [Errno -75] NetCDF: Malformed or unexpected Constraint: b'https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'

In [ ]:
ds.mean(dim="time")#.compute()#.air.data.visualize()

